In [2]:
import time
import os
import json
import pickle
import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50, VGG19
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras import losses

from matplotlib.pyplot import imread, imshow

import h5py
import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime
import nibabel as nib
import re
from collections import Counter


%matplotlib inline
%load_ext autoreload
%autoreload 1

SEED=1
random.seed(SEED)
np.random.seed(SEED)
#tf.set_random_seed(SEED)
tf.random.set_seed

K.clear_session()



In [7]:

# Get fmri ROI data
import h5py
tr_list = []
ax_length = 285
for TR in ['TR1','TR2','TR3','TR4','TR5']:
    roi_list = []
    for roi in ['RHPPA', 'LHPPA', 'RHRSC', 'LHRSC', 'RHOPA', 'LHOPA', 'LHLOC', 'RHLOC', 'RHEarlyVis', 'LHEarlyVis']:
        with h5py.File('/home/ubuntu/boldroi/ROIs/CSI1/h5/CSI1_ROIs_%s.h5' % TR, 'r') as f:
            print(list(f.keys()))
            x_roi = list(f[roi])
            x_r = np.stack(x_roi, axis=0)
            print(x_r.shape)
            x_r = np.pad(x_r, ((0, 0), (0, ax_length-x_r.shape[1])), mode='constant', constant_values=0)
            roi_list.append(x_r)

    tr_list.append(roi_list)

#x_all = np.concatenate(roi_list)
#x_all =np.squeeze(x_all)
#print(x_all.shape)
for tr_num, roi_list in enumerate(tr_list):
    for roi_num, roi in enumerate(roi_list):
        print("tr: %s, roi: %s shape: %s" % (tr_num, roi_num, str(roi.shape)))


['LHEarlyVis', 'LHLOC', 'LHOPA', 'LHPPA', 'LHRSC', 'RHEarlyVis', 'RHLOC', 'RHOPA', 'RHPPA', 'RHRSC']
(5254, 200)
['LHEarlyVis', 'LHLOC', 'LHOPA', 'LHPPA', 'LHRSC', 'RHEarlyVis', 'RHLOC', 'RHOPA', 'RHPPA', 'RHRSC']
(5254, 131)
['LHEarlyVis', 'LHLOC', 'LHOPA', 'LHPPA', 'LHRSC', 'RHEarlyVis', 'RHLOC', 'RHOPA', 'RHPPA', 'RHRSC']
(5254, 143)
['LHEarlyVis', 'LHLOC', 'LHOPA', 'LHPPA', 'LHRSC', 'RHEarlyVis', 'RHLOC', 'RHOPA', 'RHPPA', 'RHRSC']
(5254, 86)
['LHEarlyVis', 'LHLOC', 'LHOPA', 'LHPPA', 'LHRSC', 'RHEarlyVis', 'RHLOC', 'RHOPA', 'RHPPA', 'RHRSC']
(5254, 187)
['LHEarlyVis', 'LHLOC', 'LHOPA', 'LHPPA', 'LHRSC', 'RHEarlyVis', 'RHLOC', 'RHOPA', 'RHPPA', 'RHRSC']
(5254, 101)
['LHEarlyVis', 'LHLOC', 'LHOPA', 'LHPPA', 'LHRSC', 'RHEarlyVis', 'RHLOC', 'RHOPA', 'RHPPA', 'RHRSC']
(5254, 152)
['LHEarlyVis', 'LHLOC', 'LHOPA', 'LHPPA', 'LHRSC', 'RHEarlyVis', 'RHLOC', 'RHOPA', 'RHPPA', 'RHRSC']
(5254, 190)
['LHEarlyVis', 'LHLOC', 'LHOPA', 'LHPPA', 'LHRSC', 'RHEarlyVis', 'RHLOC', 'RHOPA', 'RHPPA', 'RHRS

In [8]:
xtr_list = []
for tr_num, rlist in enumerate(tr_list):
    print("tr_num: %s" % tr_num)
    xa = np.hstack(rlist)
    xtr_list.append(xa)
    print(xa.shape)

tr_num: 0
(5254, 2850)
tr_num: 1
(5254, 2850)
tr_num: 2
(5254, 2850)
tr_num: 3
(5254, 2850)
tr_num: 4
(5254, 2850)


In [9]:
x_all = np.stack(xtr_list, axis=1)
print(x_all.shape)


(5254, 5, 2850)


In [10]:

with open('CSI1_labels_all.npy', 'rb') as f:
    labels_all = pickle.load(f)

In [46]:
# Split data into train/test set
from sklearn.utils import shuffle
num_samples = x_all.shape[0]
div = int(num_samples * 0.8)
x_shuffle, y_shuffle = shuffle(x_all, labels_all, random_state=0)
x_train = x_shuffle[0:div, :]
y_train = y_shuffle[0:div]
x_test = x_shuffle[div:, :]
y_test = y_shuffle[div:]

#x_train = x_all[0:div, :]
#y_train = labels_all[0:div]
#x_test = x_all[div:, :]
#y_test = labels_all[div:]

#num_samples = x_all.shape[0]
##num_voxels = 100  # maximum number of voxels to use for training
#div = int(num_samples * 0.6)
#print("Division index: %s" % div)
##x_shuffle, y_shuffle = shuffle(x_all, y_all, random_state=0)
#x_train = x_all[0:div, :]
##y_train = y_all[0:div, :]
#y_train = labels_all[0:div]
#
#x_test = x_all[div:, :]
##y_test = y_all[div:, :]
#y_test = labels_all[div:]
print("x train shape: %s" % str(x_train.shape))
print("y train shape: %s" % str(y_train.shape))
print("x test shape: %s" % str(x_test.shape))
print("y test shape: %s" % str(y_test.shape))


x train shape: (4203, 5, 2850)
y train shape: (4203, 3)
x test shape: (1051, 5, 2850)
y test shape: (1051, 3)


In [47]:

del classifier_model
X_input = Input(shape=(x_train.shape[1], x_train.shape[2]))
#X = Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2))(X_input)
#X = TimeDistributed(Dense(32, activation='relu'))(X_input)
X = LSTM(64, dropout=0.2, recurrent_dropout=0.2)(X_input)
X = Dense(32, activation='relu')(X)
X = Dense(8, activation='relu')(X)
predictions = Dense(3, activation='softmax')(X)

#X_input = Input(shape=(x_train.shape[1], x_train.shape[2]))
##X = Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2))(X_input)
#X = LSTM(64, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(X_input)
#X = TimeDistributed(Dense(32, activation='relu'))(X)
#X = LSTM(64, dropout=0.2, recurrent_dropout=0.2)(X)
#X = Dropout(0.1)(X)
#X = Dense(8, activation='relu')(X)
#X = Dropout(0.1)(X)
#predictions = Dense(3, activation='softmax')(X)

#X_input = Input(shape=(x_train.shape[1]))
#X = Dense(4, activation='relu')(X_input)
#X = Dropout(0.2)(X)
#X = Dense(32, activation='relu')(X)
#X = Dropout(0.2)(X)
#X = Dense(32, activation='relu')(X)
#X = Dropout(0.2)(X)
#X = Dense(32, activation='relu')(X)
#X = Dropout(0.2)(X)
#predictions = Dense(3, activation='softmax')(X)

classifier_model = Model(inputs=X_input, outputs=predictions)
classifier_model.summary()


i=0
callbacks = [TensorBoard(log_dir=f'./log/{i}'), EarlyStopping(monitor='val_loss', mode='min')]

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 5, 2850)]         0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 64)                746240    
_________________________________________________________________
dense_48 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_49 (Dense)             (None, 8)                 264       
_________________________________________________________________
dense_50 (Dense)             (None, 3)                 27        
Total params: 748,611
Trainable params: 748,611
Non-trainable params: 0
_________________________________________________________________


In [48]:


from tensorflow.keras import optimizers
EPOCHS=150
callbacks = [TensorBoard(log_dir=f'./log/{i}')]
sgd = optimizers.SGD(lr=0.005, decay=1e-3, momentum=0.9, nesterov=True)
#adam = optimizers.Adam(learning_rate=0.0001, decay=1e-3, beta_1=0.9, beta_2=0.999, amsgrad=False)
adam = optimizers.Adam(learning_rate=0.0001, decay=1e-3, beta_1=0.9, beta_2=0.999, amsgrad=False)
#classifier_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
#classifier_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
#classifier_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
classifier_model.fit(x=x_train, y=y_train, shuffle=True, epochs=EPOCHS, callbacks=callbacks, validation_data=(x_test, y_test))
#classifier_model.fit(x=x_train, y=y_train, epochs=EPOCHS, callbacks=callbacks, validation_data=(x_test, y_test))



Train on 4203 samples, validate on 1051 samples
Epoch 1/150
4203/4203 [==============================] - 5s 1ms/sample - loss: 1.0887 - accuracy: 0.4059 - val_loss: 1.0715 - val_accuracy: 0.4377
Epoch 2/150
4203/4203 [==============================] - 3s 737us/sample - loss: 1.0553 - accuracy: 0.4278 - val_loss: 1.0250 - val_accuracy: 0.4367
Epoch 3/150
4203/4203 [==============================] - 3s 737us/sample - loss: 1.0197 - accuracy: 0.4356 - val_loss: 0.9900 - val_accuracy: 0.4586
Epoch 4/150
4203/4203 [==============================] - 3s 739us/sample - loss: 0.9945 - accuracy: 0.4585 - val_loss: 0.9675 - val_accuracy: 0.5014
Epoch 5/150
4203/4203 [==============================] - 3s 746us/sample - loss: 0.9724 - accuracy: 0.4854 - val_loss: 0.9486 - val_accuracy: 0.5233
Epoch 6/150
4203/4203 [==============================] - 3s 746us/sample - loss: 0.9502 - accuracy: 0.5018 - val_loss: 0.9316 - val_accuracy: 0.5528
Epoch 7/150
4203/4203 [==============================] - 3s 

4203/4203 [==============================] - 3s 738us/sample - loss: 0.7485 - accuracy: 0.6464 - val_loss: 0.8206 - val_accuracy: 0.6204
Epoch 56/150
4203/4203 [==============================] - 3s 729us/sample - loss: 0.7472 - accuracy: 0.6469 - val_loss: 0.8205 - val_accuracy: 0.6194
Epoch 57/150
4203/4203 [==============================] - 3s 737us/sample - loss: 0.7461 - accuracy: 0.6507 - val_loss: 0.8203 - val_accuracy: 0.6175
Epoch 58/150
4203/4203 [==============================] - 3s 733us/sample - loss: 0.7510 - accuracy: 0.6469 - val_loss: 0.8198 - val_accuracy: 0.6185
Epoch 59/150
4203/4203 [==============================] - 3s 735us/sample - loss: 0.7455 - accuracy: 0.6410 - val_loss: 0.8199 - val_accuracy: 0.6185
Epoch 60/150
4203/4203 [==============================] - 3s 749us/sample - loss: 0.7407 - accuracy: 0.6479 - val_loss: 0.8210 - val_accuracy: 0.6194
Epoch 61/150
4203/4203 [==============================] - 3s 758us/sample - loss: 0.7383 - accuracy: 0.6531 - val

4203/4203 [==============================] - 3s 735us/sample - loss: 0.7192 - accuracy: 0.6576 - val_loss: 0.8201 - val_accuracy: 0.6289
Epoch 110/150
4203/4203 [==============================] - 3s 753us/sample - loss: 0.7145 - accuracy: 0.6702 - val_loss: 0.8196 - val_accuracy: 0.6242
Epoch 111/150
4203/4203 [==============================] - 3s 749us/sample - loss: 0.7140 - accuracy: 0.6621 - val_loss: 0.8199 - val_accuracy: 0.6270
Epoch 112/150
4203/4203 [==============================] - 3s 747us/sample - loss: 0.7124 - accuracy: 0.6636 - val_loss: 0.8199 - val_accuracy: 0.6280
Epoch 113/150
4203/4203 [==============================] - 3s 742us/sample - loss: 0.7121 - accuracy: 0.6679 - val_loss: 0.8204 - val_accuracy: 0.6289
Epoch 114/150
4203/4203 [==============================] - 3s 743us/sample - loss: 0.7163 - accuracy: 0.6705 - val_loss: 0.8198 - val_accuracy: 0.6289
Epoch 115/150
4203/4203 [==============================] - 3s 753us/sample - loss: 0.7177 - accuracy: 0.6667